<a href="https://colab.research.google.com/github/yyaroslavskiy/cuddly-chainsaw/blob/feature%2Farina_api_yandex_geocoder/parsing/api_yandex_geocoder/api_yandex_geocoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с адрессами и API карт (геокодинг)

In [ ]:
import pandas as pd
import re
import requests
import json
from geopy.geocoders import Nominatim
from tqdm import tqdm
import logging
import time
import sys
import random

df = pd.read_csv("doctors.csv")

/tmp/ipython-input-1656099521.py:12: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("doctors.csv")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/geocoding_results/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Новый раздел

In [ ]:
df

,Unnamed: 0,name,experience,rating_sber,review_count_sber,price_sber,link_sber,clinics_count_sber,clinic_1_name_sber,clinic_1_address_sber,...,clinic_1_metro_prod,clinic_2_name_prod,clinic_2_address_prod,clinic_2_metro_prod,clinic_3_name_prod,clinic_3_address_prod,clinic_3_metro_prod,speciality,is_kids,is_adults
0,0,Абросимова Елена Владимировна,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Рассказовка,NaN,NaN,NaN,NaN,NaN,NaN,['терапевт'],False,True
1,1,Абросимова Елена Владимировна,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Пыхтино,NaN,NaN,NaN,NaN,NaN,NaN,['терапевт'],False,True
2,2,Агеева Наталья Юрьевна,18.0,4.5,0.0,3500.0,https://docdoc.ru/doctor/Ageeva_Natalya_12,1.0,Будь Здоров на Рязанском проспекте,"г. Москва, ул. Рязанский пр-т, д. 2Б",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
3,3,Агеева Наталья Юрьевна,NaN,4.0,0.0,NaN,https://docdoc.ru/doctor/Ageeva_Natalya_3,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,4,Акимова Анна Владимировна,18.0,4.3,0.0,NaN,https://docdoc.ru/doctor/Akimova_Anna_7,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38860,38522,Ященко Елена Николаевна,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Фрунзенская,NaN,NaN,NaN,NaN,NaN,NaN,"['акушер', 'детский гинеколог', 'гинеколог']",True,True
38861,38523,Ященко Ирина Анатольевна,30.0,4.7,2.0,NaN,https://docdoc.ru/doctor/Yashtenko_Irina,0.0,NaN,NaN,...,Печатники,NaN,NaN,NaN,NaN,NaN,NaN,"['реабилитолог', 'невролог']",False,True
38862,38524,Ящина Ольга Владимировна,NaN,4.0,3.0,NaN,https://docdoc.ru/doctor/Yashtina_Olga,0.0,NaN,NaN,...,Фрунзенская,NaN,NaN,NaN,NaN,NaN,NaN,"['остеопат', 'терапевт']",False,True
38863,38525,Ящук Сергей Борисович,41.0,4.3,3.0,NaN,https://docdoc.ru/doctor/Yashtuk_Sergey,0.0,NaN,NaN,...,Щукинская,NaN,NaN,NaN,NaN,NaN,NaN,"['акушер', 'гинеколог-хирург', 'гинеколог']",False,True


In [ ]:
#Воспользуемся следующей функцией, которая предоставляет отчет о пропусках в данных

def missing_values_table(df):
    """
    Функция возвращает резюме по пропущенным значениям
    """
    # Общее число пропусков
    mis_val = df.isnull().sum()

    # Процент пропусков
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Создадит таблицу с результатом
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Переименнуем колонки
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})

    # Отсортируем по проценту пропущенных значений
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)

    # Выведем некоторую информацию
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")

    return mis_val_table_ren_columns


missing_values_table(df)

Your selected dataframe has 34 columns.
There are 30 columns that have missing values.


,Missing Values,% of Total Values
clinic_3_metro_sber,38732,99.7
clinic_3_address_sber,38723,99.6
clinic_3_name_sber,38723,99.6
clinic_2_metro_sber,38038,97.9
clinic_2_address_sber,37997,97.8
clinic_2_name_sber,37997,97.8
clinic_3_metro_prod,36627,94.2
clinic_3_name_prod,36251,93.3
clinic_3_address_prod,36251,93.3
price_sber,34666,89.2


Соберем все унникальные адреса в отдельную табличку чтобы запросить их координаты через апи. Для начала приведем (попытаемся) адреса к нормализованному виду.

In [ ]:
#функция приведения адреса к нормализированному
def normalize_address(addr):

  #приводим к строке - если нан возвращаем пустую строку
    if pd.isna(addr):
        return ""

    addr = str(addr).lower().strip()

    #словарь для нормализация адресов (я умерла)
    replacements = {
    #улицы, переулки, проспекты
    'ул.': 'улица',
    'пер.': 'переулок', 'пер ': 'переулок ', 'пер-к': 'переулок',
    'пр.': 'проезд', 'пр-д': 'проезд', 'пр ': 'проезд ', 'проезд': 'проезд',
    'проспект': 'пр-т', 'просп.': 'пр-т',
    'бульвар': 'бул', 'бул.': 'бул',
    'ш.': 'шоссе', 'шоссе': 'шоссе',
    'пл.': 'площадь', 'площадь': 'площадь',
    'наб.': 'набережная', 'набережная': 'набережная',

    #дома и корпуса
    'дом': 'д.',
    'к.': 'корпус', 'корп.': 'корпус',
    'строение': 'стр', 'стр.': 'стр',

    #города
    'пос.': 'поселок', 'п.': 'поселок',
}
    for old, new in replacements.items():
        addr = addr.replace(old, new)


    addr = re.sub(r'\b(офис|помещение|этаж|вход|каб\.?|комн\.?)\b.*', '', addr)

    #убираем знаки лишние
    addr = re.sub(r'[;,]+$', '', addr)          # убрать хвостовые запятые и точки
    addr = re.sub(r'\s+', ' ', addr)            # двойные пробелы → один
    addr = re.sub(r'\s+д\.$', '', addr)

    #привести город к одному виду
    addr = re.sub(r'(^|\s)город\s+', 'г. ', addr)
    addr = re.sub(r'(^|\s)г\s*\.?\s*', 'г. ', addr)

    #удалить если в начале россия
    addr = re.sub(r'^россия,?\s*', '', addr)

    #удалить строение в конце
    addr = re.sub(r'стр\.?\s*$', '', addr)

    #удалить скобки и то что в них
    addr = re.sub(r'\(.*?\)', '', addr)

    return addr



Соберем информацию по каждому адресу из изначальной таблицы. Чтобы для каждого адреса была информация о его клинике, врачу, метро и тд

In [ ]:
#детальная таблица адресов с информацией о клиниках
address_records = []

#проходимся по адресам сбера для каждого врача чтобы собрать детальную инфу по каждому адресу
for i in [1, 2, 3]:
    sber_mask = df[f'clinic_{i}_address_sber'].notna()
    for idx, row in df[sber_mask].iterrows():
        address_records.append({
            'address': row[f'clinic_{i}_address_sber'],
            'clinic_name': row[f'clinic_{i}_name_sber'],
            'metro': row[f'clinic_{i}_metro_sber'],
            'source': 'sber',
            'doctor_name': row['name']
        })

#проходимся по адресам продокторов для каждого врача
for i in [1, 2, 3]:
    prod_mask = df[f'clinic_{i}_address_prod'].notna()
    for idx, row in df[prod_mask].iterrows():
        address_records.append({
            'address': row[f'clinic_{i}_address_prod'],
            'clinic_name': row[f'clinic_{i}_name_prod'],
            'metro': row[f'clinic_{i}_metro_prod'],
            'source': 'prodoctorov',
            'doctor_name': row['name']
        })

#переводим массив словарей в датафрейм
addresses_detailed = pd.DataFrame(address_records)

#нормализуем адреса (господи)
addresses_detailed['address_norm'] = addresses_detailed['address'].apply(normalize_address)  #ура

print("сделана детальная таблица адресов: addresses_detailed, ее размер", addresses_detailed.shape)

unique_addresses = addresses_detailed[['address','address_norm']].drop_duplicates()
#unique_addresses = unique_addresses.sort_values().reset_index(drop=True)

print(f"сделана таблица уникальных адресов для геокодинга: unique_addresses, ее размер", unique_addresses.shape)


сделана детальная таблица адресов: addresses_detailed, ее размер (42312, 6)
сделана таблица уникальных адресов для геокодинга: unique_addresses, ее размер (4776, 2)


детализированная таблица адресов (к ней потом будем возвращать координаты через `address_norm`)

In [ ]:
addresses_detailed

,address,clinic_name,metro,source,doctor_name,address_norm
0,"г. Москва, ул. Рязанский пр-т, д. 2Б",Будь Здоров на Рязанском проспекте,Нижегородская,sber,Агеева Наталья Юрьевна,"г. москва, улица рязанский пр-т, д. 2б"
1,"г. Москва, ул. Щукинская, д. 2",ЭммаКлиник,Щукинская,sber,Алексеева Екатерина Михайловна,"г. москва, улица щукинская, д. 2"
2,"г. Москва, Последний пер., д. 28",Будь Здоров на Сретенке,Сухаревская,sber,Антипова Анна Дмитриевна,"г. москва, последний переулок, д. 28"
3,"г. Москва, ул. Нагорная, д. 17, корп. 6",Юсуповская больница на Нагорной,Нагорная,sber,Аранович Элина Викторовна,"г. москва, улица нагорная, д. 17, корпус 6"
4,"г. Тверь, ул. Крылова, д. 21",Лаборатория KDLLAB на Крылова,NaN,sber,Барбашинова Юлия Владимировна,"г. тверь, улица крылова, д. 21"
...,...,...,...,...,...,...
42307,"Котельники, ул. Кузьминская, д. 7Б",Поликлиника на Кузьминской,Котельники,prodoctorov,Ярусова Инна Викторовна,"котельники, улица кузьминская, д. 7б"
42308,"ул. Энгельса, д. 7/21",«Q Клиника» на Энгельса (Лаборатория «ЛабКвест»),Бауманская,prodoctorov,Ярцева Ирина Михайловна,"улица энгельса, д. 7/21"
42309,"Реутов, Юбилейный просп., д. 80",Городская поликлиника №2 на Юбилейном проспекте,Новокосино,prodoctorov,Ясинская Светлана Борисовна,"реутов, юбилейный пр-т, д. 80"
42310,"Махачкала, ул. Ахульго, д. 8",«Дана Медикал» на Ахульго,NaN,prodoctorov,Яхьяева Байсари Исаковна,"махачкала, улица ахульго, д. 8"


Таблица уникальных ( ну или почти уникальных) адресов

In [ ]:
unique_addresses

,address_norm
0,"г. москва, улица рязанский пр-т, д. 2б"
1,"г. москва, улица щукинская, д. 2"
2,"г. москва, последний переулок, д. 28"
3,"г. москва, улица нагорная, д. 17, корпус 6"
4,"г. тверь, улица крылова, д. 21"
...,...
42292,"электролитный проезд, д. 16, корпус 2"
42298,"верея, площадь советская, д. 1"
42300,"улица люсиновская, д. 36, стр 2"
42307,"котельники, улица кузьминская, д. 7б"


In [ ]:
'''
#удалила дубликаты где адрес и название клиники одинаковое
addresses_detailed_1 = (
    addresses_detailed
    .drop_duplicates(subset=['address_norm', 'clinic_name'], keep='first')
    .sort_values('address_norm')
)

#посмотрим дубликаты адресов где почему-то разные названия клиник, видим что таких 1400
addresses_detailed_1[addresses_detailed_1.duplicated(subset=['address_norm'], keep=False)]'''

"\n#удалила дубликаты где адрес и название клиники одинаковое\naddresses_detailed_1 = (\n    addresses_detailed\n    .drop_duplicates(subset=['address_norm', 'clinic_name'], keep='first')\n    .sort_values('address_norm')\n)\n\n#посмотрим дубликаты адресов где почему-то разные названия клиник, видим что таких 1400\naddresses_detailed_1[addresses_detailed_1.duplicated(subset=['address_norm'], keep=False)]"

In [ ]:
'''
#Группируем по адресу и считаем уникальные клиники
clinics_per_address = (
    addresses_detailed_1
    .groupby('address_norm')
    .agg(
        unique_clinics=('clinic_name', 'nunique'),  # количество уникальных клиник
        clinic_names=('clinic_name', lambda x: list(x.unique())),  # список уникальных названий
        total_records=('clinic_name', 'count')  # общее количество записей
    )
    .sort_values('unique_clinics', ascending=False)
    .reset_index()
)

# Показать адреса с несколькими клиниками
multiple_clinics = clinics_per_address[clinics_per_address['unique_clinics'] > 1]
print(f"Адресов с несколькими клиниками: {len(multiple_clinics)}")
print(multiple_clinics.head(10))'''

'\n#Группируем по адресу и считаем уникальные клиники\nclinics_per_address = (\n    addresses_detailed_1\n    .groupby(\'address_norm\')\n    .agg(\n        unique_clinics=(\'clinic_name\', \'nunique\'),  # количество уникальных клиник\n        clinic_names=(\'clinic_name\', lambda x: list(x.unique())),  # список уникальных названий\n        total_records=(\'clinic_name\', \'count\')  # общее количество записей\n    )\n    .sort_values(\'unique_clinics\', ascending=False)\n    .reset_index()\n)\n\n# Показать адреса с несколькими клиниками\nmultiple_clinics = clinics_per_address[clinics_per_address[\'unique_clinics\'] > 1]\nprint(f"Адресов с несколькими клиниками: {len(multiple_clinics)}")\nprint(multiple_clinics.head(10))'

Попытка работать с опен стрит мап апи - оч плохо читает адреса, все в нанах

In [ ]:
'''
addresses = unique_addresses[0:5]

geolocator = Nominatim(user_agent="doctor_geo_parser", timeout=10)

results = []
for addr in tqdm(addresses):
  time.sleep(1)
  try:
      location = geolocator.geocode(addr)
      if location:
          results.append({
              "address": addr,
              "lat": location.latitude,
              "lon": location.longitude
          })
      else:
          results.append({"address": addr, "lat": None, "lon": None})
  except Exception as e:
      results.append({"address": addr, "lat": None, "lon": None})

geo_df = pd.DataFrame(results)
'''


'\naddresses = unique_addresses[0:5]\n\ngeolocator = Nominatim(user_agent="doctor_geo_parser", timeout=10)\n\nresults = []\nfor addr in tqdm(addresses):\n  time.sleep(1)\n  try:\n      location = geolocator.geocode(addr)\n      if location:\n          results.append({\n              "address": addr,\n              "lat": location.latitude,\n              "lon": location.longitude\n          })\n      else:\n          results.append({"address": addr, "lat": None, "lon": None})\n  except Exception as e:\n      results.append({"address": addr, "lat": None, "lon": None})\n\ngeo_df = pd.DataFrame(results)\n'

#  **API**

---



Так как не получилось брать координаты из открытой апишки, мы будем отправлять по два запроса на каждый адрес с яндекса. Первый запрос - чтобы получить точные координаты. Второй запрос - чтобы получить район и округ по координатам (просто по адресу это не получается)

In [ ]:
API_key = "ed6b1a79-a77a-498a-a3f0-8d5b3a04ccd1"

url = "https://geocode-maps.yandex.ru/v1"
address = "г. Москва, ул. Миклухо-Маклая, д. 43"

params = {
    "apikey": API_key,
    "geocode": address,
    "lang": "ru_RU",
    "format": "json",
    "results": 1
}

response = requests.get(url, params=params)
print(response.url)

https://geocode-maps.yandex.ru/v1?apikey=ed6b1a79-a77a-498a-a3f0-8d5b3a04ccd1&geocode=%D0%B3.+%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0%2C+%D1%83%D0%BB.+%D0%9C%D0%B8%D0%BA%D0%BB%D1%83%D1%85%D0%BE-%D0%9C%D0%B0%D0%BA%D0%BB%D0%B0%D1%8F%2C+%D0%B4.+43&lang=ru_RU&format=json&results=1


In [ ]:
res = response.json()

res

{'response': {'GeoObjectCollection': {'metaDataProperty': {'GeocoderResponseMetaData': {'request': 'г. Москва, ул. Миклухо-Маклая, д. 43',
     'results': '1',
     'found': '1'}},
   'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'exact',
        'text': 'Россия, Москва, улица Миклухо-Маклая, 43',
        'kind': 'house',
        'Address': {'country_code': 'RU',
         'formatted': 'Россия, Москва, улица Миклухо-Маклая, 43',
         'postal_code': '117485',
         'Components': [{'kind': 'country', 'name': 'Россия'},
          {'kind': 'province', 'name': 'Центральный федеральный округ'},
          {'kind': 'province', 'name': 'Москва'},
          {'kind': 'locality', 'name': 'Москва'},
          {'kind': 'street', 'name': 'улица Миклухо-Маклая'},
          {'kind': 'house', 'name': '43'}]},
        'AddressDetails': {'Country': {'AddressLine': 'Россия, Москва, улица Миклухо-Маклая, 43',
          'CountryNameCode': 'RU',
          'Count

In [ ]:
#пробую достать координтаты
pos = res['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
print(f"координаты: '{pos}'")

координаты: '37.523779 55.644105'


#Прямой геокодинг - получаем координаты каждого адреса


In [ ]:
#куча всего внизу - это дыкий многопоточность на запросы к апи
#нам надо почти 10к запросов обработать при лимите в тысячу

API_KEYS = [
    "759df403-9a97-4a65-958f-3bfd8221edba",
    "1dfae84d-7edd-4f69-8e9c-ce4dcb471ddf",
    "d0d8be16-9487-4b74-b978-37391c273958",
    "da17485b-5995-4d9e-bf67-54b805424f71",
    "87aac881-da92-4344-874a-10b1acf8820d",
    "c57bed78-7460-46e6-983d-c6fa7b3ea582",
    "47f64396-a850-4da5-b110-e9db4d6853c1",
    "2a1d6052-cf87-4bda-959d-91ca5ce46650",
    "d15cf053-d232-4a56-b9b1-9707c8e80f04",
    "313e3642-4bbc-42e7-8c83-4c78feb681ab",
    "f5d6cb9e-8f26-4f90-bdd7-00a181c7e634",
    "ed6b1a79-a77a-498a-a3f0-8d5b3a04ccd1"
]

#лимиты на запросы
KEY_REQUEST_LIMIT = 300   #сколько запросов безопасно делать на один ключ
KEY_SWITCH_THRESHOLD = KEY_REQUEST_LIMIT
PER_REQUEST_SLEEP = 0.5  #базовая пауза между запросами
MAX_RETRIES = 1

#глобальные переменные для подсчёта запросов
requests_per_key = 0
current_key_index = 0
MAX_REQUESTS_PER_KEY = 1000

#какой ключ активен и сколько запросов на него сделали
_current_key_index = 0
_requests_on_key = 0

logger = logging.getLogger('geocoder')

#переключатель ключа - вставлять вместо ключа
def get_api_key():
    global _current_key_index, _requests_on_key

    #не достиг лимита?#если достиг меняем ключ и сбрасываем счетчик реквестов
    if _requests_on_key >= KEY_SWITCH_THRESHOLD:
        _current_key_index = (_current_key_index + 1) % len(API_KEYS)
        _requests_on_key = 0
        logger.info("Смена API ключа: %d/%d...", _current_key_index+1, len(API_KEYS))
    _requests_on_key += 1
    return API_KEYS[_current_key_index]

#защита моего аккаунта от бана... сейф реквест вместо обычного реквеста (обработка разных ответов сервера)
#реализация ретрая при ошибках сервиса и бекоффа
def safe_request(url, params, timeout=10):

    backoff = 5 #откат по времени
    for attempt in range(1, MAX_RETRIES+1):
        try:
            resp = requests.get(url, params=params, timeout=timeout)
            status = resp.status_code

            #успешный ответ
            if status == 200:
                #небольшая случайная пауза для вариативности
                time.sleep(PER_REQUEST_SLEEP + random.uniform(0, 0.6))
                return resp

            #Too Many Requests - уважительно ждем дольше бедного бро
            if status == 429:
                logger.warning("429 received, attempt %d. Backing off %ds", attempt, backoff)
                time.sleep(backoff)
                backoff *= 2
                continue

            #серверные ошибки — пробуем повторить
            if 500 <= status < 600:
                logger.warning("%d серверная ошибка, attempt %d. Backing off %ds", status, attempt, backoff)
                time.sleep(backoff)
                backoff *= 2
                continue

            #клиентские ошибки не повторяем, логируем и возвращаем нан
            logger.error("Request failed with status %d: %s", status, resp.text[:200])
            return None

        except requests.RequestException as e:
            logger.warning(logger.warning("Сетевая ошибка %s - пробуем еще...", e))
            time.sleep(backoff)
            backoff *= 2

    logger.error("Все попытки доступа провалились сори((")
    return None


In [ ]:
def geocode_address(address):
    url = "https://geocode-maps.yandex.ru/1.x/"
    params = {
        'apikey': get_api_key(),
        'geocode': address,
        'format': 'json',
        'results': 1
    }
    response = safe_request(url, params=params)
    return response.json()

In [ ]:
#для проверки что адрес в границах москвы

def is_in_moscow_area(lon, lat):
    #примерные координаты Москвы и рядом с запасом
    #московский регион - широта 55.0–56.3, долгота 36.4–38.3
    return 36.4 <= lon <= 38.3 and 55.0 <= lat <= 56.3

In [ ]:
#функция осуществляющая прямой геокодинг - вытягивание координат

#логи
logger = logging.getLogger('direct_geocoder')
logger.setLevel(logging.INFO)
logger.propagate = False
logger.handlers = []

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(formatter)

file_handler = logging.FileHandler(f'{path}direct_geocode.log', encoding='utf-8')
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

logger.info('Запуск прямого геокодера... Готов к работе😈')

def direct_geocode(address):


    try:
        #отправляем запрос к API
        res = geocode_address(address)

        #проверяем, есть ли результаты
        members = res['response']['GeoObjectCollection']['featureMember']
        if not members:
          logger.warning(f"❗️❗️❗️ для адреса '{address}' не найдено результатов 😭")
          return {
                'address': address,
                'longitude': None,
                'latitude': None,
            }

        #извлекаем координаты
        geo_object = res['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
        lon, lat = map(float, geo_object.split())
        coords = {
        'longitude': lon,
        'latitude': lat,
        }

        #проверка на то что адрес в москве
        in_moscow = is_in_moscow_area(lon, lat)

        if in_moscow:
            logger.info(f"🟢 Успешно!'{address}' → {lon}, {lat} это в  Москве (МО)")
        else:
            #обнуляем координаты, если не Москва
            logger.warning(f"❗️⚠️ '{address}' вне Москвы ({lon}, {lat}) - координаты удалены")
            coords = {'longitude': None, 'latitude': None}

        return {
            'address': address,
            'longitude': coords['longitude'],
            'latitude': coords['latitude'],
        }

    except Exception as e:
        logger.error(f"❗️❗️❗️ Ошибка при обработке адреса '{address}': {e} 😭😭😭😭")
        return {
            'address': address,
            'longitude': None,
            'latitude': None,
        }


logger.info("Тестирование на пробных адресах👠🍌")

test_addresses = [
    "Москва, Красная площадь, 1",
    "Санкт-Петербург, Невский проспект, 1",
    "Казань, Кремль",
    "какой-то неправильный адрес🍌",
    "77"
]

results = []

for i, addr in enumerate(test_addresses):
    logger.info(f"Тест {i+1}/{len(test_addresses)}: '{addr}'")
    result = direct_geocode(addr)
    results.append(result)
    time.sleep(0.5)


logger.info("👻👻👻Тестирование завершено👻👻👻")



logger.info(f"\n🤟Прямой геокодинг готов!🤟")



2025-11-10 04:53:38,138 - INFO - Запуск прямого геокодера... Готов к работе😈
2025-11-10 04:53:38,144 - INFO - Тестирование на пробных адресах👠🍌
2025-11-10 04:53:38,146 - INFO - Тест 1/5: 'Москва, Красная площадь, 1'
2025-11-10 04:53:39,170 - INFO - 🟢 Успешно!'Москва, Красная площадь, 1' → 37.61768, 55.755277 это в  Москве (МО)
2025-11-10 04:53:39,677 - INFO - Тест 2/5: 'Санкт-Петербург, Невский проспект, 1'
2025-11-10 04:53:40,651 - WARNING - ❗️⚠️ 'Санкт-Петербург, Невский проспект, 1' вне Москвы (30.312185, 59.936846) - координаты удалены
2025-11-10 04:53:41,153 - INFO - Тест 3/5: 'Казань, Кремль'
2025-11-10 04:53:42,646 - WARNING - ❗️⚠️ 'Казань, Кремль' вне Москвы (49.11573, 55.793642) - координаты удалены
2025-11-10 04:53:43,148 - INFO - Тест 4/5: 'какой-то неправильный адрес🍌'
2025-11-10 04:53:44,126 - WARNING - ❗️⚠️ 'какой-то неправильный адрес🍌' вне Москвы (135.395445, 44.710064) - координаты удалены
2025-11-10 04:53:44,627 - INFO - Тест 5/5: '77'
2025-11-10 04:53:45,931 - WARNIN

In [ ]:
unique_addresses_df = pd.DataFrame(unique_addresses, columns=['address_norm'])
unique_addresses_df

,address_norm
0,"г. москва, улица рязанский пр-т, д. 2б"
1,"г. москва, улица щукинская, д. 2"
2,"г. москва, последний переулок, д. 28"
3,"г. москва, улица нагорная, д. 17, корпус 6"
4,"г. тверь, улица крылова, д. 21"
...,...
42292,"электролитный проезд, д. 16, корпус 2"
42298,"верея, площадь советская, д. 1"
42300,"улица люсиновская, д. 36, стр 2"
42307,"котельники, улица кузьминская, д. 7б"


In [ ]:
#прямой геокодинг по всем адресам

#логи
logger = logging.getLogger('direct_geocoder')
logger.setLevel(logging.INFO)
logger.propagate = False
logger.handlers = []

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(formatter)

file_handler = logging.FileHandler(f'{path}direct_geocode.log', encoding='utf-8')
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

logger.info("Начинаю атаку на яндекс карты (запускаю геокодинг)...")

def geocode_addresses_to_dataframe(addresses_df):

    #копия чтобы не менять оригинал
    result_df = addresses_df.copy()

    #новые колонки для координат
    result_df['latitude'] = None
    result_df['longitude'] = None

    logger.info(f"Всего адресов для обработки: {len(result_df)}")

    #обрабатываем адреса с прогресс баром
    for idx, row in tqdm(result_df.iterrows(), total=len(result_df), desc="геокодинг адресов"):
        address = row['address_norm']

        try:
            geocode_result = direct_geocode(address)

            #сохраняем результаты
            result_df.at[idx, 'latitude'] = geocode_result['latitude']
            result_df.at[idx, 'longitude'] = geocode_result['longitude']

            #статус обработки
            if geocode_result['latitude'] is not None:
                logger.info(f"🟢 Успешно: '{address}'")
            else:
                logger.warning(f"❗️❗️❗️ Не найдено: '{address}'")

        except Exception as e:
            result_df.at[idx, 'geocode_status'] = f'Ошибка: {str(e)}'
            logger.error(f"❗️ Ошибка при обработке '{address}': {e}")

         #если все слетит:
        if (idx + 1) % 150 == 0:
          result_df.to_csv(f'{path}adresses_with_coord_{idx}.csv', index=False, encoding='utf-8')
          logger.info(f"Промежуточные результаты сохранены -'adresses_with_coord_{i}.csv' ({idx+1}/{len(df)}")

        time.sleep(0.3)

    return result_df


#ЗАПУСК (СЮДА МАССИВ ПОДАВАТЬ)
addresses_coords = geocode_addresses_to_dataframe(unique_addresses_df)

#cтатистика
success_count = addresses_coords[addresses_coords['latitude'].notna()].shape[0]

logger.info(f"Результаты геокодинга:")
logger.info(f"\nВсего обработано: {len(addresses_coords)}")
logger.info(f"😍Успешно: {success_count}/{len(addresses_coords)}")

#сохраняем
addresses_coords.to_csv(f'{path}addresses_with_сoordinates.csv', index=False, encoding='utf-8')
logger.info(f"Результат сохранен в: {'addresses_with_сoordinates.csv'}")

logger.info("Успех! Прямой геокодинг завершен! Отдыхаем ")

addresses_coords.head(3)

2025-11-10 05:17:42,885 - INFO - Начинаю атаку на яндекс карты (запускаю геокодинг)...
2025-11-10 05:17:42,894 - INFO - Всего адресов для обработки: 4749


геокодинг адресов:   0%|          | 0/4749 [00:00<?, ?it/s]

2025-11-10 05:17:44,282 - INFO - 🟢 Успешно!'г. москва, улица рязанский пр-т, д. 2б' → 37.738656, 55.730487 это в  Москве (МО)
2025-11-10 05:17:44,284 - INFO - 🟢 Успешно: 'г. москва, улица рязанский пр-т, д. 2б'


геокодинг адресов:   0%|          | 1/4749 [00:01<2:13:24,  1.69s/it]

2025-11-10 05:17:45,547 - INFO - 🟢 Успешно!'г. москва, улица щукинская, д. 2' → 37.481936, 55.809641 это в  Москве (МО)
2025-11-10 05:17:45,566 - INFO - 🟢 Успешно: 'г. москва, улица щукинская, д. 2'


...

геокодинг адресов: 100%|█████████▉| 4748/4749 [2:04:19<00:01,  1.54s/it]

2025-11-10 07:22:03,573 - WARNING - ❗️⚠️ 'махачкала, улица ахульго, д. 8' вне Москвы (47.464949, 42.986443) - координаты удалены
2025-11-10 07:22:03,575 - WARNING - ❗️❗️❗️ Не найдено: 'махачкала, улица ахульго, д. 8'


геокодинг адресов: 100%|██████████| 4749/4749 [2:04:20<00:00,  1.57s/it]

2025-11-10 07:22:03,892 - INFO - Результаты геокодинга:
2025-11-10 07:22:03,893 - INFO - 
Всего обработано: 4749
2025-11-10 07:22:03,894 - INFO - 😍Успешно: 4068/4749
2025-11-10 07:22:03,950 - INFO - Результат сохранен в: addresses_with_сoordinates.csv
2025-11-10 07:22:03,952 - INFO - Успех! Прямой геокодинг завершен! Отдыхаем 


,address_norm,latitude,longitude
0,"г. москва, улица рязанский пр-т, д. 2б",55.730487,37.738656
1,"г. москва, улица щукинская, д. 2",55.809641,37.481936
2,"г. москва, последний переулок, д. 28",55.769681,37.630625


In [ ]:
addresses_coords = pd.read_csv('addresses_with_сoordinates.csv')

#Обратный геокодинг

из полученных координат

In [ ]:
def reverse_geocode(lon, lat):
    url = "https://geocode-maps.yandex.ru/1.x/"
    params = {
        'apikey': get_api_key(),
        'geocode': f"{lon},{lat}",  # долгота,широта
        'format': 'json',
        'kind': 'district',  #вот тут прописали что хотим именно кайнд район получить
        'results': 1
    }
    response = safe_request(url, params=params)
    return response.json()

In [ ]:
#задали пробные координаты
lon, lat = 37.519674, 55.641783   # Разделяем '37.7759 55.75553'
res = reverse_geocode(lon, lat)

#смотрим что вернулось
print("ответ апишки:")
print(res['response']['GeoObjectCollection']['featureMember'][0])

ответ апишки:
{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'other', 'text': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы', 'kind': 'district', 'Address': {'country_code': 'RU', 'formatted': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы', 'Components': [{'kind': 'country', 'name': 'Россия'}, {'kind': 'province', 'name': 'Центральный федеральный округ'}, {'kind': 'province', 'name': 'Москва'}, {'kind': 'locality', 'name': 'Москва'}, {'kind': 'district', 'name': 'Юго-Западный административный округ'}, {'kind': 'district', 'name': 'район Коньково'}, {'kind': 'district', 'name': '48-й–50-й кварталы'}]}, 'AddressDetails': {'Country': {'AddressLine': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы', 'CountryNameCode': 'RU', 'CountryName': 'Россия', 'AdministrativeArea': {'AdministrativeAreaName': 'Москва', 'Locality': {'LocalityName': 'Москва

In [ ]:
res

{'response': {'GeoObjectCollection': {'metaDataProperty': {'GeocoderResponseMetaData': {'Point': {'pos': '37.519674 55.641783'},
     'request': '37.519674,55.641783',
     'results': '1',
     'found': '1'}},
   'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'other',
        'text': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы',
        'kind': 'district',
        'Address': {'country_code': 'RU',
         'formatted': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы',
         'Components': [{'kind': 'country', 'name': 'Россия'},
          {'kind': 'province', 'name': 'Центральный федеральный округ'},
          {'kind': 'province', 'name': 'Москва'},
          {'kind': 'locality', 'name': 'Москва'},
          {'kind': 'district', 'name': 'Юго-Западный административный округ'},
          {'kind': 'district', 'name': 'район Коньково'},
          {'kind': 'distric

ответ имеет структуру

response → GeoObjectCollection → featureMember → [GeoObject]

далее я исследую как работать с этим ответом ваще

In [ ]:
#посмотрим на струтуру ответа апишки (матрешка блин...)

#1
print(res.keys()) #['response']

#2
print(res['response'].keys())  #['GeoObjectCollection']
print(res['response']['GeoObjectCollection'].keys()) #['metaDataProperty', 'featureMember']

#далее листы хранятся - в фичер то что мы ищем, а метаданные это мы отдали
print(type(res['response']['GeoObjectCollection']['featureMember'])) #<class 'list'>

#сотрим на элементы
print(res['response']['GeoObjectCollection']['featureMember'][0])

print(res['response']['GeoObjectCollection']['featureMember'][0]['GeoObject'])


dict_keys(['response'])
dict_keys(['GeoObjectCollection'])
dict_keys(['metaDataProperty', 'featureMember'])
<class 'list'>
{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'other', 'text': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы', 'kind': 'district', 'Address': {'country_code': 'RU', 'formatted': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы', 'Components': [{'kind': 'country', 'name': 'Россия'}, {'kind': 'province', 'name': 'Центральный федеральный округ'}, {'kind': 'province', 'name': 'Москва'}, {'kind': 'locality', 'name': 'Москва'}, {'kind': 'district', 'name': 'Юго-Западный административный округ'}, {'kind': 'district', 'name': 'район Коньково'}, {'kind': 'district', 'name': '48-й–50-й кварталы'}]}, 'AddressDetails': {'Country': {'AddressLine': 'Россия, Москва, Юго-Западный административный округ, район Коньково, 48-й–50-й кварталы', 'CountryNameCode': 'RU', 'CountryNam

In [ ]:
#посмотрим на все компоненты вернувщиеся так как у меня ничо не работает
def get_components(geo_object):
  #идем до нужного места
    components = geo_object['metaDataProperty']['GeocoderMetaData']['Address']['Components']

    #проходимся по листу
    print("Вернувшиеся компоненты адреса:")
    for i, comp in enumerate(components):
        print(f"  {i}. {comp['kind']}: '{comp['name']}'")

    return components

components = get_components(res['response']['GeoObjectCollection']['featureMember'][0]['GeoObject'])
#очень миленько узнали что округ район и кварталы хранятсия под одним типом)))))))))))))))

Вернувшиеся компоненты адреса:
  0. country: 'Россия'
  1. province: 'Центральный федеральный округ'
  2. province: 'Москва'
  3. locality: 'Москва'
  4. district: 'Юго-Западный административный округ'
  5. district: 'район Коньково'
  6. district: '48-й–50-й кварталы'


In [ ]:
#функция чтобы возвращать район (перебираю дистрикт где есть ключевые слова просто)
def get_district(components):
    for comp in components:
        if comp['kind'] == 'district' and 'район' in comp['name'].lower():
            return comp['name']
    return None

district = get_district(components)
print(f"район: {district}")

#функция чтобы возвращать округ
def get_area(components):
    for comp in components:
        if comp['kind'] == 'district' and 'округ' in comp['name'].lower():
            return comp['name']
    return None

area = get_area(components)
print(f"округ: {area}")
print(" УРАААААААА")

район: район Коньково
округ: Юго-Западный административный округ
 УРАААААААА


In [ ]:
unique_addresses_with_coords = unique_addresses.merge(
    addresses_coords,
    on='address_norm',
    how='left'
)

,address,address_norm,latitude,longitude
0,"г. Москва, ул. Рязанский пр-т, д. 2Б","г. москва, улица рязанский пр-т, д. 2б",55.730487,37.738656
1,"г. Москва, ул. Щукинская, д. 2","г. москва, улица щукинская, д. 2",55.809641,37.481936
2,"г. Москва, Последний пер., д. 28","г. москва, последний переулок, д. 28",55.769681,37.630625
3,"г. Москва, ул. Нагорная, д. 17, корп. 6","г. москва, улица нагорная, д. 17, корпус 6",55.677524,37.607709
4,"г. Тверь, ул. Крылова, д. 21","г. тверь, улица крылова, д. 21",NaN,NaN
...,...,...,...,...
4771,"Электролитный проезд, д. 16, корп. 2","электролитный проезд, д. 16, корпус 2",55.671280,37.611024
4772,"Верея, пл. Советская, д. 1","верея, площадь советская, д. 1",NaN,NaN
4773,"ул. Люсиновская, д. 36, стр. 2","улица люсиновская, д. 36, стр 2",55.722852,37.620501
4774,"Котельники, ул. Кузьминская, д. 7Б","котельники, улица кузьминская, д. 7б",55.665274,37.867529


In [ ]:
#соединяю все вместе получаю район и округ из апишки

#создаем логгер
logger = logging.getLogger('my_geocoder')
logger.setLevel(logging.INFO)

#отключаем распространение к корневому логгеру (дублируется все без этого)
logger.propagate = False

#очищаем старые обработчики
logger.handlers = []

#создаем обработчики вручную
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(formatter)

file_handler = logging.FileHandler(f'{path}reverse_geocode.log', encoding='utf-8')
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

logger.info('Запуск обратного геокодинга... Готов обрабатывать геоданные 😈')

def add_district_and_area(df):

    df = df.copy()
    df['district'] = None
    df['area'] = None

    lon, lat = None, None

    success = 0
    fail = 0

    for i, row in tqdm(df.iterrows(), total=len(df), desc="Обратный геокодинг"):

        lon, lat = row['longitude'], row['latitude']

        if pd.isna(lon) or pd.isna(lat):
            logger.warning(f"❗️⚠️ Пропущено: нет координат для строки {i} адреса {row["address_norm"]}")
            fail += 1
            df.at[i, 'district'] = None
            df.at[i, 'area'] = None
            continue

        try:
            res = reverse_geocode(lon, lat)
            components = res['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['Address']['Components']

            district = get_district(components)
            area = get_area(components)
            logger.info(f"😍 Успешно для {i}-го. Записаны {district}, {area}")
            success +=1

            df.at[i, 'district'] = district
            df.at[i, 'area'] = area

        except Exception as e:
            logger.error(f"❗️❗️❗️Ошибка при обработке {i}-го адреса {e}")
            df.at[i, 'district'] = "Ошибка"
            df.at[i, 'area'] = "Ошибка"
            fail += 1


        #если все слетит:
        if (i + 1) % 250 == 0:
              df.to_csv(f'{path}adresses_final_{i}.csv', index=False, encoding='utf-8')
              logger.info(f"Промежуточные результаты сохранены ({i+1}/{len(df)}")

        time.sleep(0.5)

    return df, success, fail

##ТУТ ЗАПУСК
remaining_df = unique_addresses_with_coords.iloc[426:]
result_df, success, fail = add_district_and_area(remaining_df)

logger.info("Результаты обратного геокодинга:")
logger.info(f"Всего строк: {len(result_df)}")
logger.info(f"😇Успешно обработано: {success}")
logger.info(f"😭 Ошибок или пропусков: {fail}")
logger.info(f"Успешность: {success / len(result_df):.1%}")

result_df.to_csv(f'{path}adresses.csv', index=False, encoding='utf-8') #сохранение с индексом и символами
logger.info("🤟Файл успешно сохранен: adresses.csv")

result_df.head()

2025-11-10 15:01:39,649 - INFO - Запуск обратного геокодинга... Готов обрабатывать геоданные 😈


Обратный геокодинг:   0%|          | 0/990 [00:00<?, ?it/s]

2025-11-10 15:01:41,471 - INFO - 😍 Успешно для 3786-го. Записаны район Выхино-Жулебино, Юго-Восточный административный округ


Обратный геокодинг:   0%|          | 1/990 [00:02<38:08,  2.31s/it]

2025-11-10 15:01:43,573 - INFO - 😍 Успешно для 3787-го. Записаны 6-й микрорайон, None


...

Обратный геокодинг: 100%|█████████▉| 989/990 [24:27<00:01,  1.41s/it]

2025-11-10 15:26:07,244 - WARNING - ❗️⚠️ Пропущено: нет координат для строки 4775 адреса махачкала, улица ахульго, д. 8


Обратный геокодинг: 100%|██████████| 990/990 [24:27<00:00,  1.48s/it]

2025-11-10 15:26:07,250 - INFO - Результаты обратного геокодинга:
2025-11-10 15:26:07,251 - INFO - Всего строк: 990
2025-11-10 15:26:07,252 - INFO - 😇Успешно обработано: 555
2025-11-10 15:26:07,254 - INFO - 😭 Ошибок или пропусков: 435
2025-11-10 15:26:07,255 - INFO - Успешность: 56.1%
2025-11-10 15:26:07,287 - INFO - 🤟Файл успешно сохранен: adresses.csv


,address,address_norm,latitude,longitude,district,area
3786,"Самаркандский бульвар, д. 17, корп. 2","самаркандский бул, д. 17, корпус 2",55.695469,37.815696,район Выхино-Жулебино,Юго-Восточный административный округ
3787,"Видное, ул. Березовая, д. 24","видное, улица березовая, д. 24",55.539803,37.712614,6-й микрорайон,None
3788,"Казань, ул. Калинина, д. 69","казань, улица калинина, д. 69",NaN,NaN,None,None
3789,"ул. Долгоруковская, д. 27, стр. 1","улица долгоруковская, д. 27, стр 1",55.776750,37.602265,Тверской район,Центральный административный округ
3790,"Краснодар, проезд Репина, д. 34","краснодар, проезд репина, д. 34",NaN,NaN,None,None


In [ ]:
result_df

In [ ]:
# у меня парсер обратный геокодер несколько раз останавливался так что из всех
#сохранившихся файлов беру инфу и дропаю дубликаты просто

import glob

#все промежуточные файлы
files = sorted(glob.glob(path + "adresses_final_*.csv"))

#+ итоговый
files.append(path + "adresses.csv")

print(f"найдено файлов: {len(files)}")

#объединяю
dfs = [pd.read_csv(f) for f in files]
merged = pd.concat(dfs, ignore_index=True)

#убираю дубли по адресу (он уникальный)
merged = merged.drop_duplicates(subset=['address_norm'], keep='last').reset_index()

merged.to_csv(path + 'adresses_the_last_last_final_version_hooraaaaay.csv', index=False, encoding='utf-8')

print(f"😇готово: {len(merged)} уникальных адресов сохранено в adresses_full_merged.csv")

найдено файлов: 19
😇готово: 4749 уникальных адресов сохранено в adresses_full_merged.csv


In [ ]:
merged

,index,address,address_norm,latitude,longitude,district,area
0,43404,"г. Москва, ул. Рязанский пр-т, д. 2Б","г. москва, улица рязанский пр-т, д. 2б",55.730487,37.738656,Нижегородский район,Юго-Восточный административный округ
1,43405,"г. Москва, ул. Щукинская, д. 2","г. москва, улица щукинская, д. 2",55.809641,37.481936,район Щукино,Северо-Западный административный округ
2,43406,"г. Москва, Последний пер., д. 28","г. москва, последний переулок, д. 28",55.769681,37.630625,Мещанский район,Центральный административный округ
3,43407,"г. Москва, ул. Нагорная, д. 17, корп. 6","г. москва, улица нагорная, д. 17, корпус 6",55.677524,37.607709,район Котловка,Юго-Западный административный округ
4,43408,"г. Тверь, ул. Крылова, д. 21","г. тверь, улица крылова, д. 21",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4744,64195,"Электролитный проезд, д. 16, корп. 2","электролитный проезд, д. 16, корпус 2",55.671280,37.611024,Нагорный район,Южный административный округ
4745,64196,"Верея, пл. Советская, д. 1","верея, площадь советская, д. 1",NaN,NaN,NaN,NaN
4746,64197,"ул. Люсиновская, д. 36, стр. 2","улица люсиновская, д. 36, стр 2",55.722852,37.620501,район Замоскворечье,Центральный административный округ
4747,64198,"Котельники, ул. Кузьминская, д. 7Б","котельники, улица кузьминская, д. 7б",55.665274,37.867529,NaN,NaN
